# Static Stability of Canoes in Water
## Buoyancy force and submergence depth
---

### Simple Case

In order to introduce the concept of buoyancy and how it can be used to calculate submergence depth, we consider the simple model of a cuboid block partly submerged in hydrostatic (still) water:

<p><center> 
<img src="figures/B_fig01.png" width="300">
</center></p> 

A cuboid with volume $V_c$ and submerged volume $V_w$. We can then determine the mass of our cube and water by multiplying their respective volumes with their densities, mass per unit volume. Let $D_c$ and $D_w$ be the densities of the cube and water. Then the mass $M_c$ and $M_w$ can be calculated as follows: 

$$
M_c = V_c \cdot D_c \quad\quad\quad M_w = V_w \cdot D_w
$$

Using Archimedes' principle, the bouyancy force is equivalent to the water's displaced weight multiplied by gravity. We should also note the force of gravity acting on our cube its mass multiplied by gravity. We calculate the forces $F_g$ and $F_b$ acting on our cube.

$$
F_g = - M_c \cdot g \quad\quad\quad F_b = M_w \cdot g
$$

The total sum of forces is our measure of if the cube will sink or rise, where positive will be treated as rising up, negative will be treated as sinking deeper. 


In [1]:
import scripts.buoyancy_interface as bi #might be fine to have in the same cell
bi.WaterLevelCubeGraph(length = 1.2, depth = 0.125, height = 1, density = 500, resolution = 1024) #add a little arrow in the future to help visualize force

interactive(children=(FloatSlider(value=0.0, description='waterHeight', max=1.0, step=0.0009765625), Output())…

You may notice there exists a point where the totale force equals zero. This is know as the buoyancy equilibrium or neutral buoyancy. It is where the object will neither sink or rise, as the force of gravity and buoyancy are equal in magnitude and cancel eachother out. For our simple cube model we can calculate the depth where this can happen.

We ideally $F_b + F_c = 0$, rearranging and expanding gives us:

$$
\begin{align}
g\cdot V_c\cdot D_c & = g\cdot V_w \cdot D_w \\
(\text{length} \cdot \text{width} \cdot \text{height}) \cdot D_c & =  (\text{length} \cdot \text{width} \cdot \text{water depth}) \cdot D_w \\
\text{height} \cdot \frac{ D_c}{D_w} & = \text{water depth}
\end{align}
$$

In [2]:
bi.EquilibriumCubeGraph(length = 1, depth = 1, height = 1, resolution = 1024) #possibly delete.

interactive(children=(FloatSlider(value=497.5263671875, description='density', max=997.0, step=0.9736328125), …

---
### General case

The volume and weight method is an intuiative approach to finding buoyancy, however calculating the submerged volume of an object isn't always easy. In order to rectify this problem we can use an important theorem in calculus called Gauss's Theorem or Divergence Theorem. It changes our problem of finding volume to one of summing over certain values along the surface of our object. 

<p><em>The volume integral of the divergence of a vector field, taken throughout a bounded domain $D$, equals the surface integral of the normal componenet of the vector field taken over the surface $S$ bounding $D$</em></p>
$$
\int\int\int_D \triangledown \cdot \mathbf{F} \text{ d}D = \int\int_S \mathbf{F}\cdot n \text{ d}S
$$
Heavy jargon aside, and to put this into terms we can understand. 

The left side's triple integral of $\triangledown \cdot \mathbf{F}$ is equivalent to summing up every unit volume multiplied by the change in pressure $F$ as you go deeper. This pressure changes linearly giving us a constant rate of change, this simplifies to density times gravity all multiplied by the volume of water displaced. This is our simple case, but cut up into infinitely small pieces. 

$$
\int\int\int_{V_w} \triangledown \cdot \mathbf{F} \text{ d}V_w =  V_w \cdot D_w \cdot g = F_b
$$

The right side is the sum of every normal vector $n$, along the surface of the water displaced, multiplied by the linear increasing water pressure $F$ as you go deeper. This is adding all the water pressure that is pushing on our object. As water pressure increases the deeper we go, the pressure force increases. [something something] This can be felt when you move your hand through water. You feel more resistance when you push against the water, versus slicing through it with your hand diagonally. Or when you dive too deep, you can feel the pressure on your ears.

<p><center>
    <img src="figures/B_fig02.png" width="600">
</center></p>
<font size = "2"><center>
    The left cube is the volume method, right cube is the normals method
</center></font>
[Make the diagram more appropriate]



In [3]:
bi.RotatedCubeGraph(length = 2) #Most likely going to remove. 

interactive(children=(FloatSlider(value=1.5707963267948966, description='theta', max=3.141592653589793, step=0…

#### General Case Approximation method
This method allows us to easily approximate the equilibrium of any model by using their surface. Since nearly all 3D surfaces are collections of triangles/ quads, we can use those to gather the neccesary data to calculate our general method. If we store the position, normals, and area in a table we can use them to calculate the pressure force for each shape. This also allows us to reuse the data multiple times as we can offset the position and get different results.

<p><center>
    <img src="figures/B_fig03.png" width="300">
</center></p>

| <center><font size="3.5">position</font>   | <center><font size="3.5">normal</font> | <center><font size="3.5">area</font> |
| --- | --- | --- |
| <center><font size="3.5">[0.1, -0.2, -3]</font></center> | <center><font size="3.5">[0, 0.5, 0.866]</font></center> | <center><font size="3.5">0.05</font></center> | 
| <center><font size="3.5">[0.2, -0.3, -3.6]</font></center> | <center><font size="3.5">[0, 0.6, 0.8]</font></center> | <center><font size="3.5">0.087</font></center> | 
| <center><font size="3.5">$\vdots$</font></center> | <center><font size="3.5">$\vdots$</font></center> | <center><font size="3.5">$\vdots$</font></center> |

<p>Summing the over the list by mutliplying each normal vector by the depth based pressure and by area gives us the total bouyancy force acting on our object.</p>
    
$$
\sum (position.z \cdot D_w \cdot g) \cdot (normal) \cdot (area) = F_b
$$
    
<p>This is a fairly inexpensive operation. It'll allow us to calculate the $F_b$ for multiple depths allowing us to search for the equilibrium level. This can be done through a binary search, using the the top and bottom of our canoe (plus some) as bounds to search through. Starting at the middle, we can determine if the total force is positive or negative, then cut our search interval in half appropriately. This will approach the equilibrium level fairly quickly. </p>

<p>There are more complex search methods that approach faster but for simplicity we will use a fairly approachable method</p>

We now have everything we need to determine the equilibrium level of our canoe [WORK ON THIS MORE]

In [4]:
bi.GenerateBoatGraph(width = 1, height = 1, length = 2.5, c = 0.1, type = 3, resolution = 32) 
#move the canoe instead!! and the numbers seem really high? Double check everything got moved over properly

interactive(children=(FloatSlider(value=5000.0, description='mass', max=10000.0, step=312.5), Output()), _dom_…

Conclusion part read other the other notebooks to see how they do theres and check with prof for any informations

# Citation

very crude at the moment, will include proper later, 


still trying to find a suitable citation for Mahdads works that I've grossily simplified.

Used Introduction to Vector Analysis, Harry F. Davis $\cdot$ A. David Snider, seventh edition. For Divergence theorem. 